In [104]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [105]:
import tensorflow as tf
import pandas as pd
import numpy as np
from preprocess import makeRawDataset
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
pd.set_option("max_rows",1000)

In [106]:
#column names in original dataset
columnNames=["AffiliationID","Place","Affiliation","AuthID","FieldID","Paper","Authors","Conference","Name","Affiliate"
             "ConfID","PaperID","Venue","Year","Topic","KeywordID","Summary","Title"]
columnNames.extend([i.lower() for i in columnNames])

In [107]:
columnNames

['AffiliationID',
 'Place',
 'Affiliation',
 'AuthID',
 'FieldID',
 'Paper',
 'Authors',
 'Conference',
 'Name',
 'AffiliateConfID',
 'PaperID',
 'Venue',
 'Year',
 'Topic',
 'KeywordID',
 'Summary',
 'Title',
 'affiliationid',
 'place',
 'affiliation',
 'authid',
 'fieldid',
 'paper',
 'authors',
 'conference',
 'name',
 'affiliateconfid',
 'paperid',
 'venue',
 'year',
 'topic',
 'keywordid',
 'summary',
 'title']

In [108]:
y_data=makeRawDataset("NLP_Dataset.csv") #provides with raw data
X=pd.read_csv("NLP_Dataset.csv").Question

In [110]:
# stop_word=set(stopwords.words("english")) | set(string.punctuation)
stop_word=set(string.punctuation)
X_without=[]
for i in X:
    temp=i.split()
    X_without.append([j for j in temp if j not in stop_word])

In [111]:
X_without

[['List', 'all', 'the', 'papers', 'IDs'],
 ['List', 'all', 'the', 'authors', 'IDs'],
 ['List',
  'all',
  'the',
  'papers',
  'which',
  'are',
  'written',
  'by',
  'author',
  'of',
  'authorID',
  '$AuthID$.'],
 ['What', 'are', 'the', 'papers', 'written', 'by', '$AuthID$?'],
 ['How',
  'many',
  'papers',
  'are',
  'written',
  'by',
  'Author',
  'ID',
  '$AuthID$?'],
 ['List',
  'all',
  'the',
  'authors',
  'who',
  'contributed',
  'to',
  'paper',
  '$PaperID$'],
 ['How', 'many', 'authors', 'contributed', 'to', 'paper', '$PaperID$'],
 ['List', 'all', 'the', 'authors', 'for', '$PaperID$'],
 ['What',
  'is',
  'the',
  'number',
  'of',
  'Authors',
  'of',
  'Paper',
  'ID',
  '$PaperID$'],
 ['List', 'all', 'the', 'papers', 'with', 'FieldID', '$FieldID$'],
 ['What',
  'are',
  'the',
  'papers',
  'associated',
  'with',
  'FieldID',
  '$FieldID$'],
 ['Count',
  'all',
  'the',
  'papers',
  'associated',
  'with',
  'FieldID',
  '$FieldID$.'],
 ['List', 'all', 'the', 'Field

In [112]:
X_correct=[]
for i in range(len(X_without)):
    temp_s=[]
    for j in range(len(X_without[i])):
        if process.extractOne(X_without[i][j],columnNames)[1]<75 and '$' not in X_without[i][j]:
            temp_s.append(X_without[i][j])
    if(len(temp_s)):
        temp_s=" ".join(i for i in temp_s)
        if(temp_s[-1] in string.punctuation):
            X_correct.append(temp_s[:-1])
        else:
            X_correct.append(temp_s)
    else:
        X_correct.append("None")
# print(len(X_correct))
# print(len(X_without))
pd.DataFrame({"X":X_without,"fuzzy":X_correct})

,X,fuzzy
0,"[List, all, the, papers, IDs]",List all the IDs
1,"[List, all, the, authors, IDs]",List all the IDs
2,"[List, all, the, papers, which, are, written, ...",List all the which are written by of
3,"[What, are, the, papers, written, by, $AuthID$?]",What are the written by
4,"[How, many, papers, are, written, by, Author, ...",How many are written by
5,"[List, all, the, authors, who, contributed, to...",List all the who contributed
6,"[How, many, authors, contributed, to, paper, $...",How many contributed
7,"[List, all, the, authors, for, $PaperID$]",List all the for
8,"[What, is, the, number, of, Authors, of, Paper...",What is the number of of
9,"[List, all, the, papers, with, FieldID, $Field...",List all the with


In [113]:
from bert_serving.client import BertClient
bc = BertClient()
X_enc=bc.encode(list(X_correct))


In [114]:
labels=[">","<","<=",">=","==","NULL","LIKE"]
encoder=LabelEncoder()
codes=encoder.fit_transform(labels)

In [115]:
codeMap={labels[i]:codes[i] for i in range(len(labels))}
codeMap

{'>': 3, '<': 0, '<=': 1, '>=': 4, '==': 2, 'NULL': 6, 'LIKE': 5}

In [116]:
inverseMap={codes[i]:labels[i] for i in range(len(labels))}

In [117]:
y=[]
for i in y_data:
    y.append(codeMap[i])

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size = 0.2, random_state = 42)

In [119]:
y_train

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 2,
 5,
 2,
 2,
 2,
 5,
 2,
 2,
 2,
 5,
 5,
 2,
 2,
 2,
 6,
 6,
 6,
 2,
 2,
 6,
 2,
 2,
 2,
 6,
 2,
 2,
 2,
 6,
 2,
 2,
 2,
 2,
 6,
 6,
 2,
 3,
 5,
 2,
 2,
 6,
 2,
 5,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 5,
 2,
 4,
 6,
 2,
 2,
 2,
 6,
 6,
 2,
 2,
 2,
 2,
 2,
 6,
 6,
 5,
 2,
 5,
 6,
 2,
 6,
 2,
 5,
 5,
 2,
 2,
 2,
 6,
 5,
 6,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 6,
 2,
 2,
 6,
 6,
 6,
 2,
 2,
 2,
 2,
 2,
 6,
 2,
 5,
 2,
 2,
 6,
 6,
 2,
 2,
 6,
 2,
 5,
 2,
 2,
 2,
 6,
 5,
 2,
 2,
 6,
 2,
 2,
 6,
 2,
 2,
 0,
 4,
 2,
 6,
 2,
 2,
 2,
 2,
 6,
 5,
 2,
 2,
 5,
 6,
 0,
 2,
 3,
 2,
 2,
 3,
 5,
 2,
 6,
 2,
 2,
 6,
 6,
 6,
 5,
 2,
 1,
 2,
 2,
 2,
 6,
 3,
 5,
 2,
 2,
 2,
 2,
 6,
 2,
 6,
 6,
 2,
 2,
 2,
 2,
 2,
 6,
 5,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 2,
 2,
 5,
 2,
 2,
 2,
 2,
 2,
 6,
 2,
 2,
 6,
 2,
 2,
 2,
 2,
 2,
 6,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 6,
 2,
 6,
 2,
 6,
 6,
 6,
 2,
 2,
 2,
 6,
 3,
 2,
 2,
 2,
 5,
 3,
 6,
 5,
 6,
 2,
 2,
 2]

In [120]:
model = XGBClassifier()
model.fit(X_train,y_train)
# model=RandomForestClassifier()
# model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [121]:
y_hat=model.predict(X_test)
# t=list(encoder.classes_)
# y_pred=encoder.inverse_transform(y_hat)
# y_true=encoder.inverse_transform(y_test)
y_pred=[]
for i in y_hat:
    y_pred.append(inverseMap[i])
y_true=[]
for i in y_test:
    y_true.append(inverseMap[i])

In [122]:
pd.DataFrame({"Original":y_true,"Predicted":y_pred})

,Original,Predicted
0,NULL,==
1,==,==
2,NULL,==
3,==,==
4,LIKE,==
5,==,==
6,==,==
7,LIKE,==
8,==,==
9,LIKE,==


In [123]:
from sklearn.metrics import roc_auc_score,auc,accuracy_score,confusion_matrix

In [124]:
roc_auc_score(y_test,y_hat)

ValueError: multiclass format is not supported

In [125]:
auc(y_test,y_hat)

ValueError: x is neither increasing nor decreasing : [6 2 6 2 5 2 2 5 2 5 2 2 2 2 2 2 2 1 2 2 6 2 4 2 2 2 5 6 2 2 2 2 6 2 6 2 5
 2 6 5 2 5 6 6 2 6 2 2 6 2 6 2 2 2 2 6 2 6 2 5 6 0 2].

In [126]:
accuracy_score(y_test,y_hat)

0.7301587301587301

In [127]:
confusion_matrix(y_test,y_hat)

array([[ 1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0],
       [ 0,  0, 33,  0,  1,  3],
       [ 0,  0,  0,  1,  0,  0],
       [ 0,  0,  7,  0,  1,  0],
       [ 0,  0,  5,  0,  0, 10]])

In [128]:
from sklearn.metrics import classification_report
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=y_hat)
print(sk_report)

              precision    recall  f1-score   support

           0   1.000000  1.000000  1.000000         1
           1   0.000000  0.000000  0.000000         1
           2   0.717391  0.891892  0.795181        37
           4   1.000000  1.000000  1.000000         1
           5   0.500000  0.125000  0.200000         8
           6   0.769231  0.666667  0.714286        15

    accuracy                       0.730159        63
   macro avg   0.664437  0.613926  0.618244        63
weighted avg   0.699713  0.730159  0.694222        63

